<img style="float: right;" src="http://www2.le.ac.uk/liscb1.jpg">
# Bioinformatics in Python

Written by Teri Forey

As well as python libraries that work with numerical data, people have also built libraries that work on biological data. Specifically DNA/protein sequences and SAM/BAM alignment files. These libraries are [Biopython](http://biopython.org/) and [pysam](http://pysam.readthedocs.io/en/latest/index.html) respectively.

## 1. Biopython

Biopython is the main python library that parses and reads sequence files. There are two main objects the `Seq` object that contains the DNA/RNA/protein sequence itself and the `SeqRecord` object that contains additional information such as an ID. We'll first look at creating a `Seq` object. 

### 1.1 Seq
We'll first import the Seq class that's contained within Bio.Seq

In [ ]:
from Bio.Seq import Seq

We can create a Seq object simply by typing in the sequence as a string.

In [ ]:
my_seq = Seq("ATGGCATCGATTCGATGCTAGGCTAGCTAG")
my_seq

When this object is printed to screen, it contains the sequence but also an Alphabet. At the moment this alphabet is just a default basic alphabet as Biopython doesn't know whether the sequence is DNA or protein. If you know this information when you create a Seq object, you should supply it.

In [ ]:
from Bio.Alphabet import generic_dna, generic_protein

my_dna = Seq("ATCGATGCAGGCTAG", generic_dna)
my_dna

In [ ]:
my_protein = Seq('ATCGATGCAGGCTAG', generic_protein)
my_protein

If these alphabets are set, it means a lot of biologically sensible errors will be caught. For example, you wouldn't be able to combine a DNA and protein sequence.

In [ ]:
my_dna + my_protein

### 1.2 Seq Methods

The Seq object contains some generic methods that are common to all strings, as well as methods that are unique to nucleotide sequences. A few examples of these are demonstrated below. 

In [ ]:
# Use the string method find() to locate the start of a particular sequence
my_dna.find('AGG')

In [ ]:
# print the reverse complement sequence
my_dna.reverse_complement()

In [ ]:
# Translate the sequence into protein. 
# Note this will only translate the forward strand and will assume the whole sequence is coding! 
my_dna.translate()

In [ ]:
# Translate will not work on a protein sequence
my_protein.translate()

### 1.3 SeqRecord

A SeqRecord object contains a Seq object, as well as additional information.

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

record = SeqRecord(Seq("MKQHKAMIVALIVICITAVVAALVTRKDLCEVHIRTGQTEVAVF",
                       IUPAC.protein),
                   id="YP_025292.1", name="HokC",
                   description="toxic membrane protein, small")
print(record)

As you can see, we've assigned the sequence, ID, name and description. Printing the record outputs this information in a human readable format.

### 1.4 SeqIO

It's unlikely you'll often create these SeqRecords by hand though, you'll most often be reading them in from a file. For that, you need to import the SeqIO function. There's an example GenBank file in `data/`, we'll import this and print a few pieces of information for each sequence in the file.

In [ ]:
from Bio import SeqIO

# We have to specify the filename and the format
for record in SeqIO.parse("../data/sequence.gb", 'genbank'):
    print("ID: %s\tName: %s\tSequence Length: %s" % (record.id, record.name, len(record.seq)))

There is only one sequence in this file, we can see all of the information by printing the whole record.

In [ ]:
print(record)

There is a lot of information here! Different file formats contain different information, so what you see here won't match what you'll see from a FASTA format file for example. Almost all of the sequence file formats you'll come across working with biological sequences are supported by Biopython, have a look at their documentation for more information.

Importantly when working with a SeqRecord object, you can access the underlying Seq object - and therefore all of the methods that come with it!

In [ ]:
record.seq

In [ ]:
record.seq.reverse_complement()

In [ ]:
# Note that to print the whole sequence you need to use print() or convert it to a string
print(record.seq)
str(record.seq)

There are lots of methods and attributes associated with a SeqRecord object, you can use the python function dir() to list them as strings. Those that start with and underscore ('_') are internal methods and shouldn't be used.

In [ ]:
dir(record)

## Exercise 1

Read in the file `data/multiple_sequences.gb`, write a for loop that will parse the contents of the genbank file outputting each sequence in FASTA format. Print a count of how many sequences are in that file.

#### Bonus 
Calculate the mean length of the sequences.

#### Bonus
Write a function that returns the N50, and use it to calculate the N50 of the sequences

In [ ]:
from Bio import SeqIO

count = 0
for record in SeqIO.parse("../data/multiple_sequences.gb", 'genbank'):
    print(record.format('fasta'))
    count += 1

print("Count: %s" % count)

#OR
#for record in SeqIO.parse("data/multiple_sequences.gb", 'genbank'):
#    print(">"+record.id)
#    print(str(record.seq))

In [ ]:
import numpy as np

def n50(lens):
    h = sum(lens)/2        # half the total length
    t = 0                  # running total
    for l in sorted(lens): # need to sort the list of sequence lengths
        t += l             # add the length to the running total
        if t >= h:         # if the running total is greater than or equal half of the total length
            return l       # return the length of the sequence (N50)
        

seqlens = []

for record in SeqIO.parse("../data/multiple_sequences.gb", 'genbank'):
    seqlens.append(len(record.seq))

print("Mean:", sum(seqlens)/len(seqlens))
print("Numpy mean:", np.mean(np.array(seqlens)))
print("N50:", n50(seqlens))

### 1.5 BioAlign

Another Biopython object that proves very useful in analysing biological data is the Align object that can store alignment data. As with SeqRecord objects you can use AlignIO.parse() to read from a file. To demonstrate other functionality we'll create an alignment by hand.

In [ ]:
from Bio.Align import MultipleSeqAlignment

aln = MultipleSeqAlignment([
             SeqRecord(Seq("ACTGCTAGCTAG", generic_dna), id="Alpha"),
             SeqRecord(Seq("ACT-CTAGCTAG", generic_dna), id="Beta"),
             SeqRecord(Seq("ACTGCTAGDTAG", generic_dna), id="Gamma"),
         ])

print(aln)

In [ ]:
# You can then print this alignment in a different format
print(aln.format('clustal'))

In [ ]:
# You can also select which rows and columns to print as you would using numpy
print(aln[:2])

print(aln[1: , 3:])

## pysam

Another library specifically written for biological data is pysam. This library is useful for reading and filtering SAM/BAM format files. 

In [ ]:
import pysam

pysam includes functions for reading in sam, cram and bam files.

In [ ]:
samfile = pysam.AlignmentFile('../data/ex1.bam', 'rb') # 'rb' stands for 'read bam', use 'rs' if the input is a sam file

You can select reads that map to a particular region in the reference using `fetch()`.

In [ ]:
for read in samfile.fetch('chr1', 100, 110):
    print(read)


You can also look over a particular region by using the `pileup()` method. In this case, you retrieve information on each column in the range provided.

In [ ]:
for pileupcolumn in samfile.pileup("chr1", 100, 120):
    print ("\ncoverage at base %s = %s" %
           (pileupcolumn.pos, pileupcolumn.n))
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            # query position is None if is_del or is_refskip is set.
            print ('\tbase in read %s = %s' %
                  (pileupread.alignment.query_name,
                   pileupread.alignment.query_sequence[pileupread.query_position]))

`pysam` even allows you to run samtools commands within python. 

*Note*: the following will only work if samtools is loaded into the virtual environment.

In [ ]:
pysam.sort("-o", "../data/ex1.sorted.bam", "../data/ex1.bam")
! ls -ltrh ../data/*.bam

## Exercise 2

Using `pysam` count the number of reads that map properly in pairs to chromosome 1.

#### Bonus
Count the reads that map perfectly (i.e. no indels or mismatches) to chromosome 1. Hint - have a look at the SAM cigar

In [ ]:
tot = 0
count = 0
for read in samfile.fetch('chr1'):
    tot += 1
    if read.is_proper_pair:
        count += 1

print("All reads:", tot)
print("Properly paired reads: {0} ({1:.2f}%)".format(count, count/tot*100))


In [ ]:
tot = 0
count =0
for read in samfile.fetch('chr1'):
    tot += 1
    if str(read.query_length) + 'M' == read.cigarstring:
        count += 1

print("All reads:", tot)
print("Properly paired reads: {0} ({1:.2f}%)".format(count, count/tot*100))